In [ ]:
import os
import time 
import pickle

import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_white"

In [ ]:
# Root of Project
ROOT = '/home/dev/ws_percept/src'

# Cameras
IMAGE_SIZE  = 240
CAMERAS = ['front', 'back']

# Scene
# [x_min, y_min, z_min, x_max, y_max, z_max] - the metric volume to be voxelized
SCENE_BOUNDS    = (-1.5, -1.5, -1.5, 1.50, 1.5, 1.5)

# Agents
AGENTS = ['Franka0', 'Franka1']

In [ ]:
# load data
DATASET_PATH = '../outputs/dualarms_2cam/dualarms_2cam_with_fk.pkl'
with open(DATASET_PATH, 'rb') as f:
    _obs = pickle.load(f)

## Setup PCD, Mesh, BB

In [ ]:
def constrain_scene_bounds(data):
    global SCENE_BOUNDS

    nX, nY, nZ, X, Y, Z = SCENE_BOUNDS

    X_mask = (data[:,0] > nX) & (data[:,0]<X)
    Y_mask = (data[:,1] > nY) & (data[:,1]<Y)
    Z_mask = (data[:,2] > nZ) & (data[:,2]<Z)
    mask = (X_mask)&(Y_mask)&(Z_mask) 

    return data[mask]

def get_pcd_and_rgb(camera_obs:dict):
    pcd = camera_obs['pointcloud'].reshape(-2,3)
    rgb = camera_obs['rgb'].reshape(-2,3)
    return pcd, rgb

def get_pcd(camera_obs:dict):
    pcd = camera_obs['pointcloud'].reshape(-2,3)
    return pcd


obs = _obs[-1]
camera_obs = obs[CAMERAS[0]]
pcd, rgb = get_pcd_and_rgb(camera_obs)

In [ ]:
from scipy.spatial.transform import Rotation as R
import cupoch as cph

ROBOT_URDF_PATH = 'outputs/testdata/franka_panda_cupoch_2/panda.urdf'
URDF_JOINT_PREFIX = 'panda_joint'

path = os.path.join(ROOT, ROBOT_URDF_PATH)
kin = cph.kinematics.KinematicChain(path)


def get_tf_matrix(
    rotation:np.array, 
    translation:np.array
) -> np.array:

  r = R.from_rotvec(rotation)
  r = r.as_matrix()

  tf = np.eye(4)
  tf[:3,:3] = r
  tf[:3,3] = translation
  
  return tf

def get_mesh_using_joint_positions_and_transform(
    joints_positions:np.array,
    global_position:np.array,
    global_rotation:np.array
):
    global URDF_JOINT_PREFIX

    # create joint_map, used for fk on urdf model
    joint_map = {'%s%d' % (URDF_JOINT_PREFIX, i+1): val for i, val in enumerate(joints_positions)} 

    # manual offset, to be removed 
    global_position += np.array([0.0, 0.0, -0.05]) 

    # create transformation matrix
    tf_matrix = get_tf_matrix(global_position, global_rotation)
    
    # do forward kinematics
    poses = kin.forward_kinematics(joint_map, tf_matrix)
    
    # store mesh geometries
    meshes = kin.get_transformed_visual_geometry_map(poses)

    return list(meshes.values())

agent_obs = obs[AGENTS[0]]
meshes = get_mesh_using_joint_positions_and_transform(
    agent_obs['joint_pos'],
    agent_obs['global_pos'],
    agent_obs['global_ang'],
)

In [ ]:
import open3d as o3d
from open3d.geometry import OrientedBoundingBox as OBB

def get_bb_array_from_mesh(mesh)->np.array:
    try:
        mesh_points = np.array(mesh.vertices.cpu())
        bb = OBB.create_from_points(
            o3d.utility.Vector3dVector(mesh_points)
        ).get_box_points()
        return np.asarray(bb)#.transpose(1,0)
    except:
        return None
    
get_bb_array_from_mesh(meshes[0])

## PCR

In [ ]:
def get_pcds_for_registration(obs):
    pcds = list()
    camera_obs = obs[CAMERAS[0]]
    pcd, rgb = get_pcd_and_rgb(camera_obs)
    pcd = constrain_scene_bounds(pcd)
    pcds.append(pcd)

    camera_obs = obs[CAMERAS[1]]
    pcd, rgb = get_pcd_and_rgb(camera_obs)
    pcd = constrain_scene_bounds(pcd)
    pcds.append(pcd)
    
    return pcds

pcds = get_pcds_for_registration(obs)

In [ ]:
def perform_point_cloud_registration(pcds):
    source = pcds[0]
    target = pcds[1]

    voxel_size = 0.05
    source_down = source.voxel_down_sample(voxel_size)
    target_down = target.voxel_down_sample(voxel_size)


In [ ]:


def do_point_cloud_registration(pcds):
    
    # load source and target pointcloud
    source_gpu = cph.geometry.PointCloud(
        cph.utility.HostVector3fVector(pcds[0])
    )
    target_gpu = cph.geometry.PointCloud(
        cph.utility.HostVector3fVector(pcds[1])
    )

    threshold = 0.02 # what does this do?
    trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0],
                            [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]])

    # register pointclouds
    start = time.time()
    reg_p2p = cph.registration.registration_icp(
        source_gpu,
        target_gpu,
        threshold,
        trans_init.astype(np.float32),
        cph.registration.TransformationEstimationPointToPlane(),
    )

    source_gpu.transform(reg_p2p.transformation)


    # remove outliers
    NEIGHBOURS = 2
    source_gpu, ind = source_gpu.remove_statistical_outlier(nb_neighbors=NEIGHBOURS, std_ratio=2.0)
    target_gpu, ind = target_gpu.remove_statistical_outlier(nb_neighbors=NEIGHBOURS, std_ratio=2.0)


    elapsed_time = time.time() - start
    print("ICP (GPU) [sec]:", elapsed_time) # adding outlier removal adds ~25ms

    # cph.visualization.draw_geometries([source_gpu+target_gpu])

    return source_gpu+target_gpu


pcd = do_point_cloud_registration(pcds)

## RBS

## Voxelization

In [ ]:
def convert_pointcloud_to_voxels(pcd):
    cubic_size = 2.0
    voxel_resolution = 100.0

    # create voxel grid
    start = time.time()
    voxels = cph.geometry.VoxelGrid.create_from_point_cloud_within_bounds(
        pcd,
        voxel_size=cubic_size / voxel_resolution,
        min_bound=(-cubic_size / 2, -cubic_size / 2, -cubic_size / 2),
        max_bound=(cubic_size / 2, cubic_size / 2, cubic_size / 2),
    )
    elapsed_time = time.time() - start
    print("Voxelization (GPU) [sec]:", elapsed_time) # adding outlier removal adds ~25ms

    # cph.visualization.draw_geometries([voxels])

    return voxels

voxels = convert_pointcloud_to_voxels(pcd)